<a href="https://colab.research.google.com/github/SamuelXJames/Signal-Denoising-Autoencoder/blob/master/CNN1D_Optimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Additional Packages to Install
# !pip install pydrive
# !pip install git+https://github.com/hyperopt/hyperopt.git
# !pip install kopt
# !pip install oauth2client
# !pip install tensorboardcolab
# !pip install pymongo
# !pip install git+https://github.com/rthalley/dnspython.git
# !pip install dnspython

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pydrive
import pymongo
import dns
from tensorboardcolab import *
from random import randint
from keras.layers import Dense,Conv1D,Input
from keras.layers import MaxPooling1D,UpSampling1D
from keras.layers import Activation,BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from kopt import CompileFN, KMongoTrials, test_fn
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.mongoexp import MongoTrials
from google.colab import drive






#MongoTrials
SRV_string  = ''
client = pymongo.MongoClient(SRV_string)
client.test_database
print(client)
hostname = ''
port = 27017





# #Authenticate/Create PyDrive client 
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

#Altrnative Saving Method
drive.mount('/content/gdrive')

  
#tbc=TensorBoardColab()
batch_size = 1000
validation_split = 0.15
timesteps = 2000
features = 1
global timesteps
global fe#atures


#Generate Test/Train Data
def create_data(timesteps = 2000):
  length = 1
  dt = 0.0005
  noise_factor = 0.35
  data_1 = []
  data_noisy_1 = []
  for i in range(batch_size):
      freq = 30*np.random.random() + 1
      t = np.arange(0,length,dt)
      #signal = np.arange(-0.5*length,0.5*length,dt)
      signal = np.sin(2*np.pi*freq*t)
      noisy_signal = signal+noise_factor*np.random.rand(len(t))
      data_1.append(signal)
      data_noisy_1.append(noisy_signal)
  data_1 = np.array(data_1)
  data_noisy_1 = np.array(data_noisy_1)
  
  #Scale Data - Need a better scaler than MinMaxScaler
  scaler = MinMaxScaler(feature_range = (0,1))
  data_1  = np.vstack((data_1,data_noisy_1))
  data_1 = scaler.fit_transform(data_1)
  signal = data_1[0:batch_size][:]
  signal_noisy = data_1[batch_size:2*batch_size][:]
  
  #Input Shaping
  features = 1
  timesteps = len(signal[0])
  shape = (batch_size,timesteps,features)
  #Input shape should equal [samples, timesteps, features]
  x = signal.reshape(shape)
  y = signal_noisy.reshape(shape)
  
  split_point = int((1-validation_split)*batch_size)
  
  x_train = x[0:split_point,:,:]
  y_train = y[0:split_point,:,:]
  x_test = x[split_point:batch_size,:,:]
  y_test = y[split_point:batch_size][:]

  return (x_train,y_train,features),(x_test,y_test)

def calculate_compressions_window_range(timesteps = 2000,window_size = 2,layers = 3,
                                       filter_spacing = None):
  window_range = []
  for i in range(layers):
    if timesteps%window_size != 0:
      if timesteps%2 == 0:
        tmp_window = 2
      elif timesteps%3 == 0:
        tmp_window = 3
      elif timesteps%5 == 0:
        tmp_window = 5
      elif timesteps%7 == 0:
        tmp_window = 7
      window_range.append(tmp_window)
      timesteps /= tmp_window
    else:
      window_range.append(window_size)
      timesteps /= window_size
    
    if filter_spacing == 'half':
      window_range = window_range[0:layers]
    
   
  return window_range 

def create_filter_range(filter_spacing,max_filters,min_filters,layers):
  #Linear Filter Range
  if filter_spacing == 'lin':
    filter_range = np.linspace(max_filters,min_filters,layers,
                               dtype = 'int64')
  #Half Filter Range
  if filter_spacing == 'half':
    if max_filters%2 != 0:
      max_filters+=1
    if min_filters%2 != 0:
      min_filters+=1
    filter_range = [max_filters]
    x = max_filters
    while x> min_filters:
      x = x/2
      filter_range.append(x)
    if filter_range[-1] < min_filters:
      del filter_range[-1]
    filter_range = np.asarray(filter_range,dtype = 'int64')
  
  #Random Filter Range
  if filter_spacing == 'random':
    filter_range = [randint(min_filters, max_filters) for z in range(layers)]
    filter_range.sort(reverse= True)
    filter_range = np.asarray(filter_range)
  
  return filter_range
  

#Gnerate Model For Optimization
def create_autoencoder(train_data,max_filters = 32, min_filters = 8, layers = 3,
                      kernel_size = 3,window_size = 2,filter_spacing = 'lin'):
  
  timesteps = 2000
  features = 1
  
  filter_range = create_filter_range(filter_spacing,max_filters,min_filters,
                                    layers)
  layers = len(filter_range)
  
  window_range = calculate_compressions_window_range(window_size = window_size,
                                                     layers = layers)
  
  
  inputs = Input(shape = (timesteps,features),name = 'Input')
  encoded = inputs
  
  
    
  conv_name = ''
  count = 1
  for num_filters in filter_range: 
    conv_name = 'Conv_Layer_No._{0}'.format(count)
    encoded = Conv1D(num_filters,kernel_size,padding = 'same',name = conv_name)(encoded)
    count+=1
    if(num_filters==max_filters):
      encoded = BatchNormalization()(encoded)
  
    encoded = Activation('relu')(encoded)
    encoded = MaxPooling1D(window_range[count-2],padding = 'same')(encoded)
  
  filter_range = np.sort(filter_range)
  for num_filters in filter_range: 
    conv_name = 'Conv_Layer_No._{0}'.format(count)
    encoded = Conv1D(num_filters,kernel_size,padding = 'same',name = conv_name)(encoded)
    count+=1
    encoded = Activation('relu')(encoded)
    encoded = UpSampling1D(window_range[-(count-layers-1)])(encoded)
    
  decoded = Conv1D(1,kernel_size,activation = 'sigmoid',padding = 'same')(encoded)
  
  autoencoder = Model(inputs,decoded)
  autoencoder.summary()
  optimizer = RMSprop(lr=1e-3)
  autoencoder.compile(optimizer = optimizer, loss = 'mse')
  
  return autoencoder





#HyperParameter Range
hyper_params = {
    "data":{
        'timesteps': 2000
        
        
    },
    "model":{
        "max_filters":hp.choice('mx_filters',np.arange(32,320,dtype = 'int32')),
        "min_filters":hp.choice('mn_filters',np.arange(4,32,dtype = 'int32')),
        'layers':hp.choice('ly',np.arange(1,3,dtype = 'int32')),
        'filter_spacing':hp.choice('fs',['lin','half'])
        
    },
    "fit":{
        'epochs': 1,
        'patience': 3
    }
}



#NN Evaluation
db_name = 'Autoencoder'
exp_name ='exp_1'
objective = CompileFN(db_name,exp_name,
                      data_fn = create_data,
                      model_fn = create_autoencoder,
                      loss_metric = 'loss',
                      loss_metric_mode = 'min',
                      valid_split = None,
                      save_model = 'best',
                      save_results = True)
                      
                      


# model_file.Upload()
# drive.CreateFile({'id': model_file.get('id')})

#test_fn(objective, hyper_params)
trials = Trials()
best = fmin(objective, hyper_params, trials=trials, algo=tpe.suggest, max_evals=2)

trial_data = trials.results
print(trial_data)

val_loss = []
loss = []
filter_spacing = []
max_filters = []
min_filters = []
layers = []
for i in range(len(trial_data)):
    trial = trial_data[i]
    val_loss.append('%.6f'%(trial['history']['loss']['val_loss'][0])) 
    loss.append('%.6f'%(trial['history']['loss']['loss'][0]))
    filter_spacing.append(trial['param']['model']['filter_spacing'])
    max_filters.append(trial['param']['model']['max_filters'])
    min_filters.append(trial['param']['model']['min_filters'])
    layers.append(trial['param']['model']['layers'])


new_dict = {'val_loss':val_loss,'loss':loss,
            'f_spacing':filter_spacing,'max_f':max_filters,'min_f':min_filters}

data_frame = pd.DataFrame.from_dict(new_dict, orient='columns')
print(data_frame)
    

#Mongo Trials - Cont'd
#trials = MongoTrials(SRV_string,exp_key = 'Optimization_1')
#trials = MongoTrials('mongo://{0}:{1}/{2}/jobs'.format(hostname,port,db_name), exp_key='exp1')
#trials = KMongoTrials(db_name, exp_name,
#                      ip=hostname,
#                      port=port)


#Save Results to Google Drive

with open('/content/gdrive/My Drive/Optimization_Results.txt', 'w') as f:
  f.write('{0}'.format(data_frame))
!cat /content/gdrive/My\ Drive/Optimization_Results.txt
# model_file = drive.CreateFile({'title' : 'Optimization_Data.txt'})
# model_file.SetContentFile(data_frame)
# model_file.Upload()
# drive.CreateFile({'id': model_file.get('id')})


 
#Save Model to Google Drive
# model_file = drive.CreateFile({'title' : 'CNN1D_v1.0.h5'})
# model_file.SetContentFile('model.h5')
# model_file.Upload()
# drive.CreateFile({'id': model_file.get('id')})
  
  
  
  
  
  